In [1]:
import pandas as  pd
import numpy as np
import warnings
import itertools
import xgboost as xgb
from tqdm import tqdm
from numpy import loadtxt
import time
warnings.filterwarnings('ignore')
kernel_with_output = True
np.random.seed(10)

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 231)
pd.set_option('display.max_columns', 100)

In [3]:
if kernel_with_output:
    df_items=pd.read_csv("../data/items.csv")
    df_categories = pd.read_csv("../data/item_categories.csv")
    df_shops = pd.read_csv("../data/shops.csv")
    df_sales = pd.read_csv("../data/sales_train.csv.gz")
    df_test = pd.read_csv("../data/test.csv.gz")
    temp = pd.read_csv("../data/sample_submission.csv.gz")


In [4]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int16)
    return df

In [5]:
df_sales["date"]=pd.to_datetime(df_sales["date"],format="%d.%m.%Y")

In [6]:
if kernel_with_output:
    # Feature engineering list
    new_features = []
    enable_feature_idea = [True, True, True, True, True, True, True, True, True, True]
    # Some parameters(maybe add more periods, score will be better) [1,2,3,12]
    lookback_range = [1,2,3,4,5,6,7,8,9,10,11,12]
    tqdm.pandas()

In [16]:
if kernel_with_output:
    current = time.time()
    trainset = pd.read_csv("../results/01/all_data.csv")
    items = pd.read_csv('../data/items.csv')
    shops = pd.read_csv('../data/shops.csv')
    # Only use more recent data
    start_month = 0
    end_month = 33
    trainset = trainset[['shop_id', 'item_id', 'item_category_id', 'date_block_num', 'item_price', 'item_cnt_month']]
    trainset = trainset[(trainset.date_block_num >= start_month) & (trainset.date_block_num <= end_month)]

    print('Loading test set...')
    test_dataset = loadtxt('../data/test.csv', delimiter="," ,skiprows=1, usecols = (1,2), dtype=int)
    testset = pd.DataFrame(test_dataset, columns = ['shop_id', 'item_id'])

    print('Merging with other datasets...')
    # Get item category id into test_df
    testset = testset.merge(items[['item_id', 'item_category_id']], on = 'item_id', how = 'left')
    testset['date_block_num'] = 34
    # Make testset contains same column as trainset so we can concatenate them row-wise
    testset['item_cnt_month'] = -1

    train_test_set = pd.concat([trainset, testset], axis = 0) 
    
    end = time.time()
    diff = end - current
    print('Took ' + str(int(diff)) + ' seconds to train and predict val set')

Loading test set...
Merging with other datasets...
Took 5 seconds to train and predict val set


In [17]:
train_test_set = downcast_dtypes(train_test_set)

#### Idea 0: Add previous shop/item sales as feature (Lag feature)

In [18]:
# add the sales of this item of this shop in i previous months, i = loopback_range
if kernel_with_output:
    if enable_feature_idea[0]:
        for diff in tqdm(lookback_range):
            feature_name = 'prev_shopitem_sales_' + str(diff)
            trainset2 = train_test_set.copy()
            trainset2.loc[:, 'date_block_num'] += diff
            trainset2.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(trainset2[['shop_id', 'item_id', 'date_block_num', feature_name]], on = ['shop_id', 'item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)
    train_test_set.head(3)


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:26<00:00,  7.91s/it]

,date_block_num,item_category_id,item_cnt_month,item_id,item_price,shop_id,prev_shopitem_sales_1,prev_shopitem_sales_2,prev_shopitem_sales_3,prev_shopitem_sales_4,prev_shopitem_sales_5,prev_shopitem_sales_6,prev_shopitem_sales_7,prev_shopitem_sales_8,prev_shopitem_sales_9,prev_shopitem_sales_10,prev_shopitem_sales_11,prev_shopitem_sales_12
0,0,40,0.0,19,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,40,0.0,19,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,40,0.0,19,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Idea 1: Add previous item sales as feature (Lag feature)

In [19]:
# add the everage of sales of an item in i previous month, ie.
# row item_id = 0, date_block_num = 21, then add a column prev_item_sales_1(the average of sales of item_id = 0 in date_block_num=20) 
if kernel_with_output:
    if enable_feature_idea[1]:
        groups = train_test_set.groupby(by = ['item_id', 'date_block_num'])
        for diff in tqdm(lookback_range):
            feature_name = 'prev_item_sales_' + str(diff)
            result = groups.agg({'item_cnt_month':'mean'})
            result = result.reset_index()
            result.loc[:, 'date_block_num'] += diff
            result.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)        
    train_test_set.head(3)


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:31<00:00,  2.81s/it]

,date_block_num,item_category_id,item_cnt_month,item_id,item_price,shop_id,prev_shopitem_sales_1,prev_shopitem_sales_2,prev_shopitem_sales_3,prev_shopitem_sales_4,prev_shopitem_sales_5,prev_shopitem_sales_6,prev_shopitem_sales_7,prev_shopitem_sales_8,prev_shopitem_sales_9,prev_shopitem_sales_10,prev_shopitem_sales_11,prev_shopitem_sales_12,prev_item_sales_1,prev_item_sales_2,prev_item_sales_3,prev_item_sales_4,prev_item_sales_5,prev_item_sales_6,prev_item_sales_7,prev_item_sales_8,prev_item_sales_9,prev_item_sales_10,prev_item_sales_11,prev_item_sales_12
0,0,40,0.0,19,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,40,0.0,19,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,40,0.0,19,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
train_test_set = downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/04/all_data_traintest_01.pkl")

#### Idea 2: Add previous shop/item price as feature (Lag feature)

In [21]:
# add the everage of the price of this item of this shop in i previous months
if kernel_with_output:
    if enable_feature_idea[2]:
        groups = train_test_set.groupby(by = ['item_id', 'shop_id', 'date_block_num'])
        for diff in tqdm(lookback_range):
            feature_name = 'prev_shopitem_price_' + str(diff)
            result = groups.agg({'item_price':'mean'}).reset_index()
            result.loc[:, 'date_block_num'] += diff
            result.rename(columns={'item_price': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(result, on = ['item_id', 'shop_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)        
    train_test_set.head(3)


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:40<00:00,  8.40s/it]

,date_block_num,item_category_id,item_cnt_month,item_id,item_price,shop_id,prev_shopitem_sales_1,prev_shopitem_sales_2,prev_shopitem_sales_3,prev_shopitem_sales_4,prev_shopitem_sales_5,prev_shopitem_sales_6,prev_shopitem_sales_7,prev_shopitem_sales_8,prev_shopitem_sales_9,prev_shopitem_sales_10,prev_shopitem_sales_11,prev_shopitem_sales_12,prev_item_sales_1,prev_item_sales_2,prev_item_sales_3,prev_item_sales_4,prev_item_sales_5,prev_item_sales_6,prev_item_sales_7,prev_item_sales_8,prev_item_sales_9,prev_item_sales_10,prev_item_sales_11,prev_item_sales_12,prev_shopitem_price_1,prev_shopitem_price_2,prev_shopitem_price_3,prev_shopitem_price_4,prev_shopitem_price_5,prev_shopitem_price_6,prev_shopitem_price_7,prev_shopitem_price_8,prev_shopitem_price_9,prev_shopitem_price_10,prev_shopitem_price_11,prev_shopitem_price_12
0,0,40,0.0,19,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,40,0.0,19,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,40,0.0,19,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
train_test_set = downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/04/all_data_traintest_012.pkl")

#### Idea 3: Add previous item price as feature (Lag feature)

In [23]:
# add the everage of the price of this item in i previous months (not care abt shops)
if kernel_with_output:
    if enable_feature_idea[3]:
        groups = train_test_set.groupby(by = ['item_id', 'date_block_num'])
        for diff in tqdm(lookback_range):
            feature_name = 'prev_item_price_' + str(diff)
            result = groups.agg({'item_price':'mean'}).reset_index()
            result.loc[:, 'date_block_num'] += diff
            result.rename(columns={'item_price': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)        
    train_test_set.head(3)


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:42<00:00,  3.58s/it]

,date_block_num,item_category_id,item_cnt_month,item_id,item_price,shop_id,prev_shopitem_sales_1,prev_shopitem_sales_2,prev_shopitem_sales_3,prev_shopitem_sales_4,prev_shopitem_sales_5,prev_shopitem_sales_6,prev_shopitem_sales_7,prev_shopitem_sales_8,prev_shopitem_sales_9,prev_shopitem_sales_10,prev_shopitem_sales_11,prev_shopitem_sales_12,prev_item_sales_1,prev_item_sales_2,prev_item_sales_3,prev_item_sales_4,prev_item_sales_5,prev_item_sales_6,prev_item_sales_7,prev_item_sales_8,prev_item_sales_9,prev_item_sales_10,prev_item_sales_11,prev_item_sales_12,prev_shopitem_price_1,prev_shopitem_price_2,prev_shopitem_price_3,prev_shopitem_price_4,prev_shopitem_price_5,prev_shopitem_price_6,prev_shopitem_price_7,prev_shopitem_price_8,prev_shopitem_price_9,prev_shopitem_price_10,prev_shopitem_price_11,prev_shopitem_price_12,prev_item_price_1,prev_item_price_2,prev_item_price_3,prev_item_price_4,prev_item_price_5,prev_item_price_6,prev_item_price_7,prev_item_price_8,prev_item_price_9,prev_item_price_10,prev_item_price_11,prev_item_price_12
0,0,40,0.0,19,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,40,0.0,19,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,40,0.0,19,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
train_test_set= downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/04/all_data_traintest_0123.pkl")

#### Idea 6: Number of month from last sale of shop/item (Use info from past)

In [25]:
# for each item of each shop, find the last i months sales. 
def create_last_sale_shop_item(row):
    for diff in range(1,33+1):
        feature_name = '_prev_shopitem_sales_' + str(diff)
        if row[feature_name] != 0.0:
            return diff
    return np.nan

In [26]:
if kernel_with_output:
    lookback_range = list(range(1, 33 + 1))
    if enable_feature_idea[6]:
        for diff in tqdm(lookback_range):
            feature_name = '_prev_shopitem_sales_' + str(diff)
            trainset2 = train_test_set[["shop_id", "item_id", "date_block_num", "item_cnt_month"]].copy()
            trainset2.loc[:, 'date_block_num'] += diff
            trainset2.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(trainset2[['shop_id', 'item_id', 'date_block_num', feature_name]], on = ['shop_id', 'item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            #new_features.append(feature_name)


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [08:01<00:00, 16.28s/it]

In [27]:
train_test_set.loc[:, 'last_sale_shop_item'] = train_test_set.progress_apply (lambda row: create_last_sale_shop_item(row),axis=1)
new_features.append('last_sale_shop_item')


  1%|▌                                                                      | 84491/11128050 [00:19<30:33, 6021.97it/s]


  2%|█▎                                                                    | 200819/11128050 [00:35<32:04, 5679.38it/s]


  3%|█▉                                                                    | 312757/11128050 [00:52<36:12, 4979.19it/s]


  4%|██▋                                                                   | 424929/11128050 [01:08<35:17, 5055.41it/s]


  5%|███▍                                                                  | 548004/11128050 [01:24<20:32, 8584.36it/s]


...

 97%|██████████████████████████████████████████████████████████████████  | 10804845/11128050 [26:36<00:57, 5632.09it/s]


 98%|██████████████████████████████████████████████████████████████████▍ | 10880531/11128050 [26:51<00:52, 4719.73it/s]


 99%|███████████████████████████████████████████████████████████████████ | 10970176/11128050 [27:05<00:20, 7652.84it/s]


100%|███████████████████████████████████████████████████████████████████▊| 11090516/11128050 [27:21<00:04, 7645.78it/s]


100%|████████████████████████████████████████████████████████████████████| 11128050/11128050 [27:28<00:00, 6749.05it/s]

In [28]:
train_test_set = downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/04/all_data_traintest_01236.pkl")

#### Idea 7: Number of month from last sale of item(Use info from past)

In [29]:
train_test_set_temp = train_test_set[["item_id", "date_block_num", "item_cnt_month"]]

In [30]:
def create_last_sale_item(row):
    for diff in range(1,33+1):
        feature_name = '_prev_item_sales_' + str(diff)
        if row[feature_name] != 0.0:
            return diff
    return np.nan

In [31]:
if kernel_with_output:
    lookback_range = list(range(1, 33 + 1))
    if enable_feature_idea[1]:
        groups = train_test_set_temp.groupby(by = ['item_id', 'date_block_num'])
        for diff in tqdm(lookback_range):
            feature_name = '_prev_item_sales_' + str(diff)
            result = groups.agg({'item_cnt_month':'mean'}).reset_index()
            result.loc[:, 'date_block_num'] += diff
            result.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)        


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [02:47<00:00,  5.79s/it]

In [32]:
train_test_set.loc[:, 'last_sale_item'] = train_test_set.progress_apply (lambda row: create_last_sale_item(row),axis=1)


  5%|███▏                                                                 | 518136/11128050 [00:22<05:39, 31239.73it/s]


  9%|██████▍                                                             | 1045179/11128050 [00:37<04:35, 36596.36it/s]


 14%|█████████▍                                                          | 1544700/11128050 [00:53<05:40, 28113.16it/s]


 18%|████████████▍                                                       | 2040110/11128050 [01:08<04:28, 33908.30it/s]


 23%|███████████████▋                                                    | 2558511/11128050 [01:23<03:42, 38507.53it/s]


...

 93%|██████████████████████████████████████████████████████████████▍    | 10371375/11128050 [05:45<00:35, 21496.77it/s]


 96%|████████████████████████████████████████████████████████████████▏  | 10661524/11128050 [06:01<00:24, 19029.26it/s]


 98%|██████████████████████████████████████████████████████████████████▍ | 10872563/11128050 [06:16<00:33, 7642.55it/s]


100%|██████████████████████████████████████████████████████████████████▉| 11123779/11128050 [06:30<00:00, 30278.04it/s]


100%|███████████████████████████████████████████████████████████████████| 11128050/11128050 [06:32<00:00, 28331.63it/s]

In [33]:
train_test_set = downcast_dtypes(train_test_set)
train_test_set.to_pickle("../results/04/all_data_traintest_012367.pkl")

In [7]:
train_test_set = pd.read_pickle("../results/04/all_data_traintest_012367.pkl")

In [8]:
if kernel_with_output:
    current = time.time()

    baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num'] +  new_features + ['item_cnt_month']

    # Clipping to range 0-20
    train_test_set['item_cnt_month'] = train_test_set.item_cnt_month.fillna(0).clip(0,20)

    # train: want rows with date_block_num from 0 to 31
    train_time_range_lo = (train_test_set['date_block_num'] >= 0)
    train_time_range_hi =  (train_test_set['date_block_num'] <= 32)

    # val: want rows with date_block_num from 22
    validation_time =  (train_test_set['date_block_num'] == 33)

    # test: want rows with date_block_num from 34
    test_time =  (train_test_set['date_block_num'] == 34)


    # Retrieve rows for train set, val set, test set
    cv_trainset = train_test_set[train_time_range_lo & train_time_range_hi]
    cv_valset = train_test_set[validation_time]
    cv_trainset = cv_trainset[baseline_features]
    cv_valset = cv_valset[baseline_features]
    testset = train_test_set[test_time]
    testset = testset[baseline_features]

    # Prepare numpy arrays for training/val/test
    cv_trainset_vals = cv_trainset.values.astype(int)
    trainx = cv_trainset_vals[:, 0:len(baseline_features) - 1]
    trainy = cv_trainset_vals[:, len(baseline_features) - 1]

    cv_valset_vals = cv_valset.values.astype(int)
    valx = cv_valset_vals[:, 0:len(baseline_features) - 1]
    valy = cv_valset_vals[:, len(baseline_features) - 1]

    testset_vals = testset.values.astype(int)
    testx = testset_vals[:, 0:len(baseline_features) - 1]

    print('Fitting...')
    model = xgb.XGBRegressor(max_depth = 11, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 1000, seed = 1, nthread = 16)
    model.fit(trainx, trainy, eval_metric='rmse')


    preds = model.predict(valx)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)

Fitting...
[22:23:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:23:21] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=11, min_child_weight=0.5, missing=None, n_estimators=100,
       n_jobs=1, nthread=16, num_round=1000, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=1, silent=None, subsample=1, verbosity=1)

In [9]:
    from sklearn.metrics import mean_squared_error
    print('val set rmse: ', np.sqrt(mean_squared_error(valy, preds)))

    preds = model.predict(testx)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)
    df = pd.DataFrame(preds, columns = ['item_cnt_month'])
    df['ID'] = df.index
    df = df.set_index('ID')
    df.to_csv('../results/04/04.csv')
    print('test predictions written to file')

    end = time.time()
    diff = end - current
    print('Took ' + str(int(diff)) + ' seconds to train and predict val, test set')

val set rmse:  1.0085536977623224
test predictions written to file
Took 541 seconds to train and predict val, test set


In [11]:
import gc
gc.collect()

24